# The Real code

In [1]:
using Random, Distributions, Plots, XLSX

What needs to be done:
. Statistics along the way
. Rebalancing ?
. ArrivalProfiles
. Last simulation
... Plots
... Time to run consecutively (does that have something to do with rebalancing?)
... 

Questions for later:
. a total number of runs (a single run has the length of the number of days per run) ??
. Struct for e?
. The runtime, and rebalancing time
... Travel time, mister vi ikke biler?? Hvordan skal man gøre det
... Pretty much just rebalancing!
. what to do when travel time passes total time?

In [2]:
### Initial function to load lambda and mu from the ArrivalProfiles.xlsx

# Change the type to Float64 for the loaded vectors
function type_change(v)
    v_ny = Vector{Float64}()
    for i in 1:length(v)
        if typeof(v[i]) == String
            push!(v_ny,parse(Float64,v[i]))
        elseif typeof(v[i]) == Int64
            push!(v_ny,float(v[i]))
        else
            push!(v_ny,v[i])
        end
    end
    return v_ny
end

function load_arri(path="C:/Users/rasmu/Desktop/OneDrive/OneDrive - Danmarks Tekniske Universitet/Decisions under uncertainty/Projekt/ArrivalProfiles.xlsx")
    xf = XLSX.readxlsx(path)

    lambda_port = 1 ./ type_change(xf[1]["C2:AA2"])
    lambda_dtu = 1 ./ type_change(xf[1]["C3:AA3"])

    mu_rammer_dtu = type_change(xf[1]["C7:AA7"])
    mu_rammer_port = type_change(xf[1]["C8:AA8"])

    return lambda_port, lambda_dtu, mu_rammer_dtu, mu_rammer_port
end


load_arri (generic function with 2 methods)

In [3]:
# Using tuples instead of PriorityQueue, because they can have multiple identical keys and the queue cannot.
# This is slightly slower but not by that much, not really a noticable difference in this case. 
# You have to get real up there in iterations before it really chugs behind.

# mutable struct Events
#     e::Array{Tuple{Float64,SubString},1}
# end
# Figure out if this can be a struct !!!

struct Environment # make struct with vectors instead
    lambda_dtu::Vector{Float64}  #arrival rate
    mu_rammer_dtu::Vector{Float64} #service rate
    lambda_port::Vector{Float64}
    mu_rammer_port::Vector{Float64}
    
    lambda_port, lambda_dtu, mu_rammer_dtu, mu_rammer_port = load_arri()
    Environment() = new(lambda_port, lambda_dtu, mu_rammer_dtu, mu_rammer_port)
end

mutable struct Stats # Make tuples to simplify functions, you are a god damn genus baby boy
    n_cars_st1::Int64
    n_cars_st2::Int64
    unfilled_st1::Int64
    unfilled_st2::Int64
    filled_st1::Int64
    filled_st2::Int64
    empty_st1::Float64
    empty_st2::Float64
end

# Might not be just as fast as PriorityQueue but it is not that much behind
function insert_and_sort!(vect::Vector, x)
    (splice!(vect, searchsorted(vect,x), [x]); vect) 
end



insert_and_sort! (generic function with 1 method)

In [4]:
# Do this:
# stats.overview_usage = [stats.overview_usage; t state.queue_length state.server_busy]
# Real nice, steal that shit!

function time_change(e::Array{Tuple{Float64,String},1},k::Int64,day::Int64)
    if k == 25
        k = 0
        day += 25
    end
    popfirst!(e)[1]

    insert_and_sort!(e,(k+day,"time_change"))
    
    k += 1
    
    return k,day
end

function arrival(station::String,e::Array{Tuple{Float64,String},1},env::Environment,stat::Stats,k::Int64)
    # WHAT to do when travel time passes total time?? makes sense? ask teacher
    
    if station == "st1"
        insert_and_sort!(e,(rand(Exponential(env.lambda_dtu[k]))+e[1][1],"st1"))
        # Student arrives at empty station. That is one unsatisfied costumer :(
        if stat.n_cars_st1 == 0
            stat.unfilled_st1 += 1
            popfirst!(e)
        else
            stat.n_cars_st1 -= 1
            stat.filled_st1 += 1
            
            insert_and_sort!(e,(rand(Exponential(env.mu_rammer_port[k]))+e[1][1],"que_st2")) 

            if stat.n_cars_st1 == 0 # station becomes empty
                stat.empty_st1 -= popfirst!(e)[1] # Empty station timer starts 
            else
                popfirst!(e)
            end
        end

    elseif station == "st2"
        insert_and_sort!(e,(rand(Exponential(env.lambda_port[k]))+e[1][1],"st2"))
        if stat.n_cars_st2 == 0
            stat.unfilled_st2 += 1
            popfirst!(e)
        else
            stat.n_cars_st2 -= 1
            stat.filled_st2 += 1
            
            insert_and_sort!(e,(rand(Exponential(env.mu_rammer_dtu[k]))+e[1][1],"que_st1") )

            if stat.n_cars_st2 == 0 # station becomes empty
                stat.empty_st2 -= popfirst!(e)[1] # Empty station timer starts 
            else
                popfirst!(e)
            end

        end

    else
        throw(ErrorException("Something went wrong in sim()."))
    end

end 

function travel(station::String,e::Array{Tuple{Float64,String},1},env::Environment,stat::Stats)

    if station == "que_st1" # If car arrives at st2
        if stat.n_cars_st1 == 0 # Arriving to an empty station without 
            stat.n_cars_st1 += 1
            stat.empty_st1 += popfirst!(e)[1] # End empty timer
        else
            stat.n_cars_st1 += 1
            popfirst!(e)    
        end
    elseif station == "que_st2" # If car arrives at st2
        if stat.n_cars_st2 == 0 # Arriving to an empty station
            stat.n_cars_st2 += 1 
            stat.empty_st2 += popfirst!(e)[1] # End empty timer
        else
            stat.n_cars_st2 += 1
            popfirst!(e)    
        end
    else 
        throw(ErrorException("Something went wrong in sim()."))
    end

end


function rebalance(desired_n_st1::Int64,desired_n_st2::Int64,stat::Stats,env::Environment,e::Array{Tuple{Float64,String},1},k::Int64)

    if stat.n_cars_st2 >= desired_n_st1
        for i in 1:desired_n_st1
            insert_and_sort!(e,(rand(Exponential(env.mu_rammer_dtu[k]))+e[1][1],"que_st1") ) 
            stat.n_cars_st2 -= 1
        end
        
        # Start empty timer
        if stat.n_cars_st2 == 0 
            stat.empty_st2 -= e[1][1]
        end

    else
        for i in 1:stat.n_cars_st2
            insert_and_sort!(e,(rand(Exponential(env.mu_rammer_dtu[k]))+e[1][1],"que_st1") )
            stat.n_cars_st2 -= 1
        end
        stat.empty_st2 -= e[1][1]

    end

    # For station 1
    if stat.n_cars_st1 >= desired_n_st2
        for i in 1:desired_n_st2
            insert_and_sort!(e,(rand(Exponential(env.mu_rammer_port[k]))+e[1][1],"que_st2") )
            stat.n_cars_st1 -= 1
        end
        
        # Start empty timer
        if stat.n_cars_st1 == 0 
            stat.empty_st1 -= e[1][1]
        end
    else
        for i in 1:stat.n_cars_st1
            insert_and_sort!(e,(rand(Exponential(env.mu_rammer_port[k]))+e[1][1],"que_st2") )
            stat.n_cars_st1 -= 1
        end
        stat.empty_st1 -= e[1][1]
    end 

    popfirst!(e)
end


rebalance (generic function with 1 method)

In [41]:
function sim(days_to_run::Float64, env::Environment,stat::Stats)
    
    e = Array{Tuple{Float64,String},1}()
    
    runs = 0
    k = 1
    day = 0
    insert_and_sort!(e,(rand(Exponential(env.lambda_dtu[k])),"st1")) # Insert first arrival
    insert_and_sort!(e,(rand(Exponential(env.lambda_port[k])),"st2")) # Insert frist arrival
    insert_and_sort!(e,(0.0,"time_change")) # THIS NEEDS TO BE FIXED!!!!!!!!

    while e[1][1] <= days_to_run*24 && runs <= 10000
        load_type = e[1][2]
        if load_type == "st1" || load_type == "st2"
            arrival(load_type,e,env,stat,k)
        elseif load_type == "que_st1" || load_type == "que_st2"
            travel(load_type,e,env,stat)
        elseif load_type == "time_change"
            k,day = time_change(e,k,day)
        elseif load_type == "rebalance"
            rebalance(desired_n_st1,desired_n_st2,stat,env,e,k)
        else
            throw(ErrorException("Issue in the event type!"))
        end
        runs += 1 # Checks the amount of events
    end
    
    # Make sure the empty timer stops at the end of the run
    if stat.n_cars_st1 == 0
        stat.empty_st1 += days_to_run*24
    elseif stat.n_cars_st2 == 0
        stat.empty_st2 += days_to_run*24
    end
    
    return stat.n_cars_st1,stat.n_cars_st2,stat.unfilled_st1,stat.unfilled_st2,stat.filled_st1,stat.filled_st2,stat.empty_st1,stat.empty_st2,runs
end

sim (generic function with 1 method)

### Testing

In [65]:
env = Environment()
stat = Stats(20,20,0,0,0,0,0,0)
sim(3.0,env,stat)

(9, 29, 261, 0, 154, 145, 21.39743927733408, 0.0, 930)

0

In [10]:
# function arrivals_generator(env::Environment)
#     e = Array{Tuple{Float64,String},1}()
#     # Arrivals at station 1
#     push!(e,(rand(Exponential(env.lambda)),"st1")) # First arrival
#     while last(e)[1] < env.hours
#        push!(e,(rand(Exponential(env.lambda))+last(e)[1],"st1"))
#     end
#     # Fix den sidste den er over 2

#     pop!(e)
#     last_st1 = last(e)[1] # Storing last time (for later use to time being empty)

#     # Arrivals at station 2 (repeat of station 1)
#     push!(e,(rand(Exponential(env.lambda)),"st2"))
#     while last(e)[1] < env.hours
#         push!(e,(rand(Exponential(env.lambda))+last(e)[1],"st2"))
#     end

#     pop!(e)
#     last_st2 = last(e)[1]

#     # Sorting the arrivals so first time is 1 and so on
#     sort!(e)

#     return e, last_st1, last_st2
# end


Environment([0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0  …  15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0], [0.1, 10.0, 10.0, 10.0, 10.0, 0.2, 0.1, 0.03333333333333333, 0.016666666666666666, 0.025  …  0.1, 0.1, 0.1, 0.2, 0.2, 10.0, 10.0, 10.0, 0.1, 0.1], [10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333  …  0.02, 0.025, 0.025, 0.05, 0.05, 0.1, 0.1, 0.1, 0.2, 10.0], [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.55, 0.55, 0.55, 0.55  …  0.55, 0.6, 0.65, 0.6, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5], [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.55, 0.6, 0.6, 0.55  …  0.55, 0.55, 0.55, 0.55, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5])